In [36]:
#BIBLIOTEKI I OPCJE
import pandas as pd
import re
import random
import math
import uuid
import datetime
import numpy as np
from tabulate import tabulate

import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

#TABELE
# playerTable = pd.DataFrame(columns=['Name', 'Surname', 'City', 'Category']) #to provide player names
playerTable = pd.read_csv('examplePlayerTable.csv')
df = pd.read_csv('examplePlayerTable.csv')
trickTable = pd.read_csv('trickTableJapan.csv')
df = df.loc[df.index <= 112]
#MODUŁY
%run tableFunction.ipynb
%run gameFunction.ipynb

[[['Bolesława', 'Karliczka', 'Lublin', 'Beginner'],
  ['January', 'Srokosz', 'Bydgoszcz', 'Advance'],
  ['Helena', 'Bąk', 'Gdańsk', 'Advance'],
  ['Marcel', 'Trznadelek', 'Poznań', 'Beginner'],
  ['Kiliana', 'Kos', 'Szczecin', 'Advance'],
  ['Natalia', 'Strzyżyk', 'Białystok', 'Beginner'],
  ['Elwira', 'Kraska', 'Katowice', 'Advance'],
  ['Lilianna', 'Kowalik', 'Karków', 'Advance'],
  ['Hilary', 'Ostrygojad', 'Wrocław', 'Advance']],
 [['Manuel', 'Alka', 'Warszawa', 'Advance'],
  ['Oskar', 'Cyranka', 'Warszawa', 'Beginner'],
  ['Damian', 'Bekasik', 'Wrocław', 'Advance'],
  ['Zoja', 'Rokitniczka', 'Łódź', 'Advance'],
  ['Ewald', 'Uhla', 'Katowice', 'Advance'],
  ['Elżbieta', 'Zięba', 'Szczecin', 'Advance'],
  ['Włodzimierz', 'Krakwa', 'Warszawa', 'Advance'],
  ['Kordian', 'Osetnik', 'Wrocław', 'Advance'],
  ['Hipolit', 'Batalion', 'Łódź', 'Beginner']],
 [['Bożena', 'Syczek', 'Lublin', 'Advance'],
  ['Hieronim', 'Remiz', 'Lublin', 'Beginner'],
  ['Eryk', 'Pardwa', 'Gdańsk', 'Advance'],
  